In [5]:
!pip install --quiet scanpy

In [23]:
import numpy as np
import pandas as pd
import scanpy as sc
from pathlib import Path

In [15]:
TEMPDIR = Path('/scratch/peidli/scPerturb/')

In [100]:
info = pd.read_csv('/home/peidli/projects/scPerturb/website/datavzrd/scperturb_dataset_info_datavzrd.csv', index_col=0)

obs_summary = pd.read_csv(TEMPDIR / 'obs/obs_tables_summary.csv', index_col=1)
# input = [f for f in (TEMPDIR / 'obs').glob('*_summary.csv')]
# obs_summary = pd.concat([pd.read_csv(f, index_col=0) for f in input], axis=1).T.set_index('dataset')

annotated_info = pd.concat([info, obs_summary], axis=1)

annotated_info = annotated_info.rename({
    'n_cells': 'Total Number of Cells',
    'n_perturbations': 'Total Number of Perturbations',
    'n_genes': 'Median Number of Genes Expressed per Cell',
    'n_counts': 'Median Number of UMI Counts per Cell',
    'n_maxperts': 'Highest Number of Perturbations in one Cell',
    'n_cells_per_perturbation': 'Median Number of Cells per Perturbation',
    'n_cells_per_perturbation_min': 'Smallest Number of Cells per Perturbation',
    'n_cells_per_perturbation_max': 'Highest Number of Cells per Perturbation'
}, axis=1)

In [135]:
f =  open('scperturb_dataset_info_datavzrd_annotated.csv')
lines = f.readlines()
with open('scperturb_dataset_info_datavzrd_annotated_cleaned.csv', 'w') as new_file:
    for line in lines:
        new_file.write(line.replace('.0,', ',').replace('.0\n', '\n'))
f.close()

In [ ]:
annotated_info.head()

In [116]:
annotated_info.index.name = 'Full index'

In [110]:
# adata = sc.read_h5ad(TEMPDIR / 'SantinhaPlatt2023.h5ad')
adata = sc.read_h5ad(TEMPDIR / 'prepared/tmp_ATAC_data_Liscovitch-BrauerSanjana2021_K562_1_feature_gene_scores.h5')

In [111]:
adata

AnnData object with n_obs × n_vars = 8723 × 23127
    obs: 'BlacklistRatio', 'nDiFrags', 'nFrags', 'PromoterRatio', 'ReadsInBlacklist', 'ReadsInPromoter', 'ReadsInTSS', 'sample', 'TSSEnrichment', 'guide', 'sgCounts', 'sgTotal', 'sgSpec', 'perturbation', 'ReadsInPeaks', 'FRIP', 'disease', 'cancer', 'tissue_type', 'cell_line', 'organism', 'perturbation_type', 'nperts', 'percent_mito', 'percent_ribo', 'ncounts', 'ngenes'
    var: 'ncounts', 'ncells', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p', 'pca'
    obsm: 'X_pca', 'X_pca_gene_scores'
    varm: 'PCs'

In [112]:
df = adata.obs.copy()
# also compute summary
df_summary = pd.Series(index=[])
df_summary['dataset'] = 'SantinhaPlatt2023'
df_summary['n_cells'] = df.shape[0]
df_summary['n_perturbations'] = df['perturbation'].nunique() - 1
df_summary['n_genes'] = df['ngenes'].median()
df_summary['n_counts'] = df['ncounts'].median()
df_summary['n_maxperts'] = df['nperts'].dropna().astype(int).max()
df_summary['n_cells_per_perturbation'] = df['perturbation'].value_counts().median()
df_summary['n_cells_per_perturbation_min'] = df['perturbation'].value_counts().min()
df_summary['n_cells_per_perturbation_max'] = df['perturbation'].value_counts().max()

/tmp/ipykernel_82/1646570234.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_summary = pd.Series(index=[])


In [113]:
df_summary

dataset                         SantinhaPlatt2023
n_cells                                      8723
n_perturbations                                21
n_genes                                    1135.0
n_counts                                  10000.0
n_maxperts                                      1
n_cells_per_perturbation                    291.5
n_cells_per_perturbation_min                   82
n_cells_per_perturbation_max                  606
dtype: object

In [115]:
pd.read_csv(TEMPDIR / 'obs_tables_summary.csv', index_col=0)

,dataset,n_perturbations,n_genes,n_counts,n_maxperts,n_cells_per_perturbation,n_cells_per_perturbation_min,n_cells_per_perturbation_max
n_cells,,,,,,,,
5768,AdamsonWeissman2016_GSM2406675_10X001,8,2580.0,9969.000000,2,555.0,6,1769
15006,AdamsonWeissman2016_GSM2406677_10X005,19,4304.0,25082.000000,4,13.0,1,1715
65337,AdamsonWeissman2016_GSM2406681_10X010,113,3690.0,15355.000000,2,554.0,1,6010
119071,AissaBenevolenskaya2021,3,49.0,75.000000,1,29874.0,29340,29983
42277,ChangYe2021,3,5278.0,18961.000000,1,7657.0,5920,21043
5905,DatlingerBock2017,96,2713.0,6711.000000,4,49.0,3,1320
39194,DatlingerBock2021,40,179.0,460.000000,2,809.0,213,4497
51898,DixitRegev2016,1728,3080.0,13974.000000,4,1.0,1,4224
218331,FrangiehIzar2021_RNA,248,3314.0,10988.000000,1,659.0,12,57605
